# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [295]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from citipy import citipy

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [308]:
city_data = pd.read_csv("../output_data/cities.csv", dtype="object", encoding="utf-8")
city_data

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,mar del plata,-38.0,-57.56,55.99,81,2,6.93,AR,2020-05-10
1,pochutla,15.74,-96.47,84.0,79,4,1.79,MX,2020-05-10
2,labuan,5.28,115.24,91.4,70,75,3.36,MY,2020-05-10
3,butaritari,3.07,172.79,83.8,76,16,6.96,KI,2020-05-10
4,puerto ayora,-0.74,-90.35,75.99,94,0,3.0,EC,2020-05-10
...,...,...,...,...,...,...,...,...,...
539,varkkallai,8.73,76.71,86.0,79,75,3.36,IN,2020-05-10
540,sambava,-14.27,50.17,75.52,81,94,14.12,MG,2020-05-10
541,zapolyarnyy,69.42,30.81,32.0,97,90,10.29,RU,2020-05-10
542,pandan,14.05,124.17,87.64,66,71,5.97,PH,2020-05-10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [325]:
# Customize the size of the figure
locations = city_data[["Latitude", "Longitude"]].astype(float)
humidity = city_data["Humidity"].astype(float)

In [326]:
figure_layout = {
    'width': '950px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(46.0, -5.0), zoom_level=3)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Ideal Weather Conditions:
- A max temperature lower than 80 degrees but higher than 70.
- Wind speed less than 10 mph.
- Zero cloudiness.
- Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [311]:
filtered_data = city_data[(city_data["Max Temperature"].astype(float)>70) & (city_data["Max Temperature"].astype(float)<80) & (city_data["Cloudiness"].astype(float)<1) & (city_data["Wind Speed"].astype(float)<10)]
filtered_data

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
4,puerto ayora,-0.74,-90.35,75.99,94,0,3.0,EC,2020-05-10
35,carnarvon,-24.87,113.63,77.45,51,0,7.78,AU,2020-05-10
37,najran,17.49,44.13,71.6,49,0,3.18,SA,2020-05-10
175,oranjemund,-28.55,16.43,70.66,34,0,4.7,NaN,2020-05-10
388,murgab,37.5,61.97,71.6,56,0,4.7,TM,2020-05-10
405,bilma,18.69,12.92,78.96,14,0,6.4,NE,2020-05-10


In [312]:
clean_data = filtered_data.dropna()
clean_data

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
4,puerto ayora,-0.74,-90.35,75.99,94,0,3.0,EC,2020-05-10
35,carnarvon,-24.87,113.63,77.45,51,0,7.78,AU,2020-05-10
37,najran,17.49,44.13,71.6,49,0,3.18,SA,2020-05-10
388,murgab,37.5,61.97,71.6,56,0,4.7,TM,2020-05-10
405,bilma,18.69,12.92,78.96,14,0,6.4,NE,2020-05-10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [313]:
# geocoordinates
hotel_coords = []
hotel_names = []
hlat=[]
hlng=[]

try:
    for index, row in clean_data.iterrows():

        target_coordinates = f"{row['Latitude']}, {row['Longitude']}"
        target_radius = 5000
        target_type = "lodging"

        # set up a parameters dictionary
        params = {
            "location": target_coordinates,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
        }

        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params).json()
        if response["results"]!=[]:
            lat = response["results"][0]["geometry"]["location"]["lat"]
            lng = response["results"][0]["geometry"]["location"]["lng"]
            hotel_name = response["results"][0]["name"]
            hlat.append(lat)
            hlng.append(lng)
            hotel_names.append(hotel_name)
            
except:
    pass

In [314]:
# Put hotel info in a dataframe
hotel_df = pd.DataFrame({"Hotel Name" : hotel_names,
                         "Lat": hlat,
                         "Lng": hlng})
hotel_df

,Hotel Name,Lat,Lng
0,Finch Bay Galapagos Hotel,-0.752802,-90.310194
1,Hospitality Carnarvon,-24.892341,113.655034
2,محمد هادي ال فطيح,17.489915,44.128800
3,Expeditions Ténére voyages,18.692979,12.917652


In [315]:
# Find city and country for each hotel
lat_lngs = zip(hotel_df["Lat"], hotel_df["Lng"])
cities = []
countries = []

# Identify nearest city for each lat, lng combination
try:
    for lat_lng in lat_lngs:
        if lat_lng[0]=='N/A' or lat_lng[1]=='N/A':
            cities.append('N/A')
            countries.append('N/A')
        else:
            city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
            country = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
            cities.append(city)
            countries.append(country)
            
except:
    pass

In [316]:
# Add cities and countries to the hotel dataframe
hotel_df["City"]=cities
hotel_df["Country"]=countries
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Finch Bay Galapagos Hotel,-0.752802,-90.310194,puerto ayora,ec
1,Hospitality Carnarvon,-24.892341,113.655034,carnarvon,au
2,محمد هادي ال فطيح,17.489915,44.128800,najran,sa
3,Expeditions Ténére voyages,18.692979,12.917652,bilma,ne


In [317]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [320]:
# Add marker layer ontop of heat map
list_of_infoboxes = []
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…